# Selecionador de seguidores - Parte 1:

Após instalar as bibliotecas necessárias, siga executando uma célula por vez, após anterior finalizar

In [1]:
import asyncio
from pyppeteer import launch
import pandas as pd
import requests
from json import loads
import time
import random
from datetime import datetime

random.seed(datetime.now())

# se ainda não tiver instalado, descomente abaixo
# !pip install pyppeteer
# !pip install pandas
# !pip install requests

# Inicialização do browser para o usuário logar

Faça login até chegar na página inicial do seu linkedin


In [2]:
browser = await launch(
        headless=False,
        args=['--no-sandbox'],
        autoClose=False
    )
page = await browser.newPage()
await page.goto("https://www.linkedin.com/login")

# digite se login e senha no browser e siga até estar na sua página inicial

# Acessa a página de seguidores para montar os headers da API autenticada

As vezes essa execução dá errado, pois não é determinística. Se der tudo certo, deve printar '8' no final. Caso contrário execute novamente. Se der erro por vezes repetidas, experimente rolar a barra de navegação até o final manualmente na tela de seguidores enquanto essa célula executa.

In [3]:
await page.goto("https://www.linkedin.com/feed/followers/")
global headers
headers = {}
await page.setRequestInterception(True)
async def intercept(request):
    if ("https://www.linkedin.com/voyager/api/feed/richRecommendedEntities" in request.url ):
        global headers
        headers = request.headers
        print(len(headers))
    await request.continue_()
page.on('request', lambda req: asyncio.ensure_future(intercept(req)))
await page.evaluate( "() => { window.scrollBy(0, window.innerHeight); }");
time.sleep(5)
cookie1 = await page.cookies()
s = []
for e in cookie1:
    s.append(f"{e['name']}={e['value']}")
cookie1_str = "; ".join(s)
headers = {key: str(value) for key, value in headers.items()}
headers["cookie"] = cookie1_str

8


# funções que consultam e tratam os dados dos seguidores via API

A api do linkedin tem paginação e retorna um json complexo, por isso são necessárias funções de extração crua, refinamento e filtragem dos dados dos seguidores para serem apresentados em uma tabela

In [49]:
def get_followers_data(countw, index_start, headers):
    try:
        url = "https://www.linkedin.com/voyager/api/feed/richRecommendedEntities?count={countw}&q=followerRecommendations&start={index_start}"
        resp = requests.get(url=url.format(countw=countw, index_start=index_start), headers=headers)
        if resp.status_code == 200:
            data = loads(resp.content.decode("utf-8"))
            return data
        else:
            raise Exception(f"not allowed. headers or coockies problems. Status: {resp.status_code}")
    except Exception as e:
        print(f"error: {e}")
        return []
    
def refine_follower_data(data):
    folowers = {}
    for element in data['data']['elements']:
        for inc in data['included']:
            ek = element.get("*recommendedEntity", "a").split(":member:")[-1]
            ik = inc.get('entityUrn',"b").split(":")[-1]
            if ek == ik:
                e = dict()
                e.update(element)
                e.update(inc)
                folowers[ek] = folowers.get(ek, {})
                folowers[ek].update(e)
    return folowers

def filter_follower_data(fl):
    return {
        "seguidores": fl["followerCount"],
        "nome": f"{fl['firstName']} {fl['lastName']}",
        "cargo": fl["occupation"],
        "link": f'https://www.linkedin.com/in/{fl["publicIdentifier"]}'
    }

def get_followers(count, index_start, headers):
    data = get_followers_data(count, index_start, headers)
    followers = refine_follower_data(data)
    filtered_followers = [filter_follower_data(fl) for fl in followers.values()]
    return pd.DataFrame.from_dict(filtered_followers)
    

# Parte que executa a consulta aleatória de fato.

Se quiser um grupo mais diverso, recomendo executar diversas consultas trazendo 1 por vez. Porem, se quiser trazer vários em uma única consulta, basta aumentar a quantidade. Não sei como a API agrupa os seguidores, mas acredito que tenha relação com o número de seguidores deles e também pela data que começaram a nos seguir.

A quantidade de seguidores definida não precisa ser exata, mas se for muito abaixo, tende a deixar de fora seus seguidores que possuem mais seguidores (não sei o porque. Não encontrei nenhuma documentação e tudo foi descoberto na tentativa e erro).


In [50]:
url_api_seguidores = "https://www.linkedin.com/voyager/api/feed/richRecommendedEntities?count={quantidade}&q=followerRecommendations&start={index_escolhidos}"
quantidade = 2
total_aproximado_de_seguidores = 11500
index_escolhidos = random.randint(0, total_aproximado_de_seguidores)

fs = get_followers(quantidade, index_escolhidos, headers)

fs

,seguidores,nome,cargo,link
0,1174,Edilaine Gropo,Commercial Manager,https://www.linkedin.com/in/edilaine-gropo
1,424,Lays Nogueira,Analista de Sistemas Sênior | Accenture Brasil,https://www.linkedin.com/in/laysnogueira
